In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define paths to the directories containing your modules
src_path_1 = r'C:\Users\mokht\Desktop\GAN-Drug-Generator\my_implementations\src'
src_path_2 = r'C:\Users\mokht\Desktop\GAN-Drug-Generator\src'

# Append paths to sys.path if they are not already included
for path in [src_path_1, src_path_2]:
    if path not in sys.path:
        sys.path.append(path)

# Now import your modules
from my_Vocabulary import Vocabulary
from my_AE import Autoencoder as AE
from utils import *

In [2]:
file_path_vae_train = r'C:\Users\mokht\Desktop\GAN-DRUG-GENERATOR\my_implementations\data\clean_smiles.csv'
df_vae_train = pd.read_csv(file_path_vae_train)
df_vae_train.head()

,SMILES
0,Cc1cc(NC(=O)COC(=O)COc2ccc3c4c(c(=O)oc3c2)CCC4...
1,Cn1c2c([N+](=O)[O-])cccc2c(=O)c2c(O)cc3c(c21)C...
2,C=C[C@]1(C)CC[C@H]2C(=C[C@@H]3OC(=O)[C@]4(C)[C...
3,COc1cc(CC[C@H](C[C@@H](OC(C)=O)[C@@H]2CCCCC[C@...
4,Cn1ccoc1=Nc1ccc(Cl)c(Cl)c1


In [3]:
smiles = df_vae_train['SMILES'].tolist()

vocab_file_path = r'C:\Users\mokht\Desktop\GAN-Drug-Generator\my_implementations\src\vocab.txt'
vocab = Vocabulary(vocab_file_path, max_len=100)
vocab.update_vocab(smiles)
tok_smiles, _ = vocab.tokenize(smiles)
print(tok_smiles[2])
X_train = vocab.encode(tok_smiles)
print(X_train[2])
len_X_train = len(X_train)
X_train = np.reshape(X_train, (len_X_train, vocab.max_len, 1))

Number of unique characters in the vocabulary: 79
['G', 'C', '=', 'C', '[C@]', '1', '(', 'C', ')', 'C', 'C', '[C@H]', '2', 'C', '(', '=', 'C', '[C@@H]', '3', 'O', 'C', '(', '=', 'O', ')', '[C@]', '4', '(', 'C', ')', '[C@H]', '3', '[C@]', '2', '3', 'C', 'C', '[C@@]', '4', '(', 'O', ')', 'O', 'C', '3', ')', 'C', '1', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
[22, 19, 16, 19, 35, 7, 2, 19, 3, 19, 19, 34, 8, 19, 2, 16, 19, 32, 9, 25, 19, 2, 16, 25, 3, 35, 10, 2, 19, 3, 34, 9, 35, 8, 9, 19, 19, 33, 10, 2, 25, 3, 25, 19, 9, 3, 19, 7, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]


In [4]:
# Autoencoder setup and loading
path_model = r'C:\Users\mokht\Desktop\GAN-Drug-Generator\models\AE\Exp9model2256_500000_biLSTM2_units512_dec_layers2-128-0.9-adam-0.1-256'
latent_dim = 256
lstm_units = 512
batch_norm = True
batch_norm_momentum = 0.9
noise_std = 0.1
numb_dec_layer = 2
emb_dim = 256

vocab_size = vocab.vocab_size  # Get the vocab size from your Vocabulary instance
decoder_input_shape = (vocab.max_len, vocab_size)
output_dim = vocab_size  # Updated based on vocabulary size

# Initialize the autoencoder with updated parameters
autoencoder = AE(path_model, decoder_input_shape, latent_dim, lstm_units, output_dim, batch_norm, batch_norm_momentum, noise_std, numb_dec_layer, emb_dim, vocab_size, vocab.max_len)
autoencoder.load_autoencoder_model(f'{path_model}\\model--86--0.0013.hdf5')

# Encode the data into latent vectors
X_train = vocab.one_hot_encoder(tok_smiles)  # Ensure data is one-hot encoded
x_latent = autoencoder.smiles_to_latent_model.predict(X_train)

# Optionally, check the shape and some entries of the latent vectors
print("Shape of the latent vectors:", x_latent.shape)
print("Example latent vectors:", x_latent[:5])

Model: "Autoencoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 smiles_to_latent_model (Functi  (None, 256)         3181568     ['encoder_inputs[0][0]']         
 onal)                                                                                            
                                                                                                  
 decoder_inputs (InputLayer)    [(None, 100, 79)]    0           []                               
                                                                                                  
 latent_to_states_model (Functi  [(None, 512),       534528      ['smiles_to_latent_mode

ValueError: Cannot assign value to variable ' embedding/embeddings:0': Shape mismatch.The variable shape (79, 256), and the assigned value shape (45, 256) are incompatible.